In [31]:
# 연습문제 : 청와대 국민청원 게시판 크롤링하기
# http://www1.president.go.kr/petitions

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  # pip install numpy 를 해야 사용가능함
import pandas as pd  # pip install pandas 해야 사용가능하고 numpy 가 먼저 설치되어 있어야 함
import xlwt

import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *100)
print("연습문제 : 청와대 국민청원 게시판 크롤링하기")
print("=" *100)

query_txt = '청와대국민청원'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\data\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('http://www1.president.go.kr/petitions')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

#시간을 계산하기 위해 현재 타임스탬프 찍기
now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d:%02d:%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)

print("\n")
page_cnt = math.ceil(cnt / 15)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

# 소주제 1: 전체 게시글의 URL 주소를 추출하여 목록을 생성하기
#Step 4. 각 게시글의 상세 정보를 수집합니다

click_count = 1

# 각 게시글의 URL 주소를 추출하여 목록을 먼저 만든 후 상세 크롤링을 진행합니다.
# URL 수집하여 목록 만들기
url_list=[]
count = 0
while ( click_count <= page_cnt + 1)  :
    
    print("%s 번째 페이지에서 게시글의 URL 정보를 추출합니다==============" %click_count)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    result_1 = soup.find_all('div','board text')
    result_2= result_1[2].find('div', class_='b_list category b_list2') 
    result_url_list=result_2.find('div','bl_body').find('ul','petition_list').find_all('li')
        
    for i in result_url_list:
        url = i.find('a')['href']
        print(url)
        url_list.append(url)
            
        count += 1

        if count == cnt :
            break  

    print("%s 번째 페이지까지 %s건 정보 수집 완료 ====================" %(click_count,count))
    print("\n")
    
    click_count += 1

    if click_count > page_cnt :
        break
    else :
        driver.find_element_by_link_text("%s" %click_count).click()
    time.sleep(3)
        
# 소주제 2: 목록을 활용하여 실제 데이터 수집하기
# Step 5. 각 게시글 별 상세 내역을 수집합니다
no = 1

cno2=[]
title2=[]
people2=[]
category2=[]
s_date2=[]
e_date2=[]
content2=[]

for i in range(0,len(url_list)):  
            
        print("\n")
        print("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no)
        
        full_url = 'http://www1.president.go.kr/'+ url_list[i]
        driver.get(full_url)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        content_all = soup.find('div','petitionsView_left_pg')
        
        f = open(ff_name, 'a',encoding='UTF-8')
        
        # 청원 등록 번호
        c_no = url_list[i].split('/')
        #cno_1 = c_no[4].split('?')
        cno_1 = c_no[2]        
        print('1.청원글번호: ',cno_1 ,'\n')
        f.write('\n')
        f.write("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no + '\n')
        f.write('1.청원번호:' + cno_1 + '\n')
        cno2.append(cno_1)
        
        # 청원제목 추출
        title = content_all.find('h3','petitionsView_title').get_text()
        print('2.청원제목: ' , title , '\n')
        f.write('2.청원제목:' + title + '\n')
        title2.append(title)
        
        # 참여인원 추출
        people = content_all.find('h2','petitionsView_count').find('span','counter').get_text()
        print('3.참여인원: ' , people ,'명' ,'\n')
        f.write('3.참여인원:' + people + '\n')
        people2.append(people)
    
        # 카테고리
        cat_all = content_all.find('div','petitionsView_info').find('ul','petitionsView_info_list').find_all('li')
        
        category = cat_all[0].get_text().replace('카테고리','')
        print('4.카테고리: ',category ,'\n')
        f.write('4.카테고리:' + category + '\n')
        category2.append(category)
        
        # 청원시작일
        s_date = cat_all[1].get_text().replace('청원시작','')
        print('5.청원시작일: ',s_date ,'\n')
        f.write('5.청원시작일:' + s_date + '\n')
        s_date2.append(s_date)
        
        # 청원마감일
        e_date = cat_all[2].get_text().replace('청원마감','')
        print('6.청원마감일: ',e_date ,'\n')
        f.write('6.청원종료일:' + e_date + '\n')
        e_date2.append(e_date)
        
        # 청원내용
        content = content_all.find('div','View_write').get_text().replace('\n','')
        print('7.청원내용: ', content.strip() , '\n')
        f.write('7.청원내용:' + content.strip() + '\n')
        content2.append(content)
        
        f.close( )
        
        no += 1
        
        if no > cnt :
            break
            
# Step 6. 출력 결과를 저장하기
# 출력 결과를 표(데이터 프레임) 형태로 만들기

blue_house = pd.DataFrame()

blue_house['청원번호'] = cno2
blue_house['청원제목'] = pd.Series(title2) 
blue_house['참여자'] = pd.Series(people2)
blue_house['카테고리'] = pd.Series(category2)
blue_house['청원시작일'] = pd.Series(s_date2)
blue_house['청원종료일'] = pd.Series(e_date2)
blue_house['청원내용'] = pd.Series(content2)
       
# csv 형태로 저장하기
blue_house.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
blue_house.to_excel(fx_name , index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

#Step 8. 요약정보 출력하기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

연습문제 6-2 : 청와대 국민청원 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 58
2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\data\):c:\data\


크롤링 할 총 페이지 번호:  4
1 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/579631
/petitions/579630
/petitions/579629
/petitions/579628
/petitions/579627
/petitions/579626
/petitions/579625
/petitions/579624
/petitions/579623
/petitions/579622
/petitions/579621
/petitions/579620
/petitions/579619
/petitions/579618
/petitions/579617
1 번째 페이지까지 15건 정보 수집 완료 ====================


2 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/579616
/petitions/579615
/petitions/579614
/petitions/579613
/petitions/579612
/petitions/579611
/petitions/579610
/petitions/579609
/petitions/579608
/petitions/579607
/petitions/579606
/petitions/579605
/petitions/579604
/petitions/579603
/petitions/579602
2 번째 페이지까지 30건 정보 수집 완료 ====================


3 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/579601
/petitions/579600
/petitions/579599
/petitions/579598
/petitions/579597
/petitions/579596
/petition

1.청원글번호:  579628 

2.청원제목:  교사가 학생을 지도할 수 있게 대책을 세워주세요!!! 

3.참여인원:  208 명 

4.카테고리:  육아/교육 

5.청원시작일:  2019-04-18 

6.청원마감일:  2019-05-18 

7.청원내용:  저는 중학교 교사입니다. 너무 답답한 마음에 이글을 씁니다. 학생들 중에 교사의 지도를 전혀 따르지 않는 학생들이 있습니다. 수업 중에 책상 위를 밟고 다닌다거나너무 떠들어서 조용히 하라고 하면 욕설을 합니다. 학생들은 욕설을 하면서 교사가 욕설을 하면 신고한다고 하거나학생 인권 운운하면서 전혀 지도를 듣지 않습니다. 문제는 이 학생들 때문에 피해를 받는 선량한 학생들입니다.수업이 전혀 진행되지 않을 때도 많고 이 학생들에게 맞거나 괴롭힘을 당하는 학생들도 많습니다. 최근에 벌점도 없어지는 추세고 얼차려 같은 신체적인 벌도 안되고 반성문을 쓰는 일이 다인데 반성문을 써도 개선되지 않고 반성문을 쓰라고 해도 무시하고 가는 경우가 대부분입니다. 미국처럼 말썽을 부리면 바로 반에서 학생들을 분리하는 폴리스도 없고 교실 밖으로 내보내는 것도 허용되지 않습니다. 부모님께 인계하는 것도 허용되지 않고 하루종일 이 학생들과 싸우면서 학교에 데리고 있으려니 교육은 다 무너졌습니다. 욕설을 하고 전혀 말을 듣지 않고 죽일 듯이 노려보며 고함을 치고 친구들이나 때리며 함부로 구는 학생들은 도대체 어떻게 지도해야 하나요. 하루종일 불안해 하며 같은교실에서 생활해야 하는 학생들이 너무 불쌍합니다. 교사가 학생을 지도할 수 있게 구체적인 대책을 세워주십시오. 정말 부탁드립니다. 



5 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  579627 

2.청원제목:  농민 및 일반인80여명을 상대로한 사기피해를 호소합니다.(실명) 

3.참여인원:  119 명 

4.카테고리:  기타 

5.청원시작일:  2019-04-18 

6.청원마감일:  2019-05-18 


1.청원글번호:  579622 

2.청원제목:  진주 '계획형 방화·살인사건'에 초기 부실한 대처로 예견된 사건을 막지 못한 경찰들 및 관련자들의 엄중한 수사를 부탁드립니다. 

3.참여인원:  81,422 명 

4.카테고리:  기타 

5.청원시작일:  2019-04-18 

6.청원마감일:  2019-05-18 

7.청원내용:  [사건 내용]2019년 4월 17일 새벽 진주 가좌동의 한 아파트에서 '계획형 방화·살인사건'이 발생하였습니다.용의자 안 씨는 당일 새벽 4시 29분쯤 휘발유를 이용해 자신의 집에 불을 지르고는 '불이야!'하고 온 주민을 깨웠습니다. 그 후 자신의 집이 있는 4층이 아닌 2층에 내려와 숨어서, 복도를 통해 대피하는 이웃을 흉기로 위협하고 살해하여 5명이 숨지고 10여명이 부상을 입었습니다. 피해자는 대부분 어린이와 노약자이고, 상대하기 어려운 덩치 큰 남성 주민은 공격하지 않았습니다. 또한 범행 후 '다 죽였다!'라고 소리치기도 했습니다. 이는 단순히 우발적인 범행 또는 '묻지마 범죄'가 아닌 약자를 대상으로 한 계획적인 범죄 행위였음을 시사합니다.[막을 수 있는 사건이었다]안 씨는 평소 이웃에게 난폭한 행동을 일삼아 올 해에만 여러 차례 경찰에 신고되었습니다. 7건의 신고 중 4건은 안 씨 집 위층 주민 강모(54)씨와 최모(18)씨가 하였으며, 증거가 없다는 이유로 적절한 보호조치를 받지 못 하였습니다. 심지어 안 씨와 '도저히 대화가 되지 않는다'는 이유로 특별한 조치 없이 돌아간 일이 있습니다. [첨부링크1]이 후 경찰의 권고로 CCTV를 설치하여 안 씨가 행한 위협적인 행위들이 입증 되었고 [첨부링크2, CCTV 영상] 불과 사건 일주일 전인 4월 11일에 경찰이 수사하였으나, 이번 참사가 일어나기 전까지 경찰은 안 씨의 정신병력을 알지 못하였습니다. [첨부링크3]경찰은 뒤늦게 안 씨의 치료 전력과 과거의 또 다른 범죄 행위를 알게 되었지만 이번 참사로 이미 최모(18)씨는 사망하였고 강모(54)씨 또한 위독한 상태입

1.청원글번호:  579616 

2.청원제목:  적폐청산 

3.참여인원:  136 명 

4.카테고리:  교통/건축/국토 

5.청원시작일:  2019-04-18 

6.청원마감일:  2019-05-18 

7.청원내용:  지난 정부의 적폐척결은 정부가 국민에게 이 나라 밝은 미래와 희망을 안겨주는 가장 큰 선물입니다. 1.  한국철도시설공단이 건설하는 충주선 철도(전철) 가칭 가남역은 대로와 연접하고 있는데도 이 공단은 2012년 ‘공사비를 조작한 허위 공문서’를 만들고 이를 빌미로 ‘이 역 광장과 연접한 대로에서 역 광장을 직접 통행하는 것을 의도적으로 차단’하면서 역광장을‘수백m 우회 통행하는 도로 1개’와 ‘꾸불꾸불 돌고 또한 오르락내리락 통행하는 도로 2개’건설을 결정하였고,또한 ‘이렇게 하면서 50억원 이상으로 추정되는 혈세를 낭비’하는 것을 지금도 고집하고 있으며 이는 의도가 있는 범죄이고 또한 나라를 말아먹는 범죄입니다. 2012년 결정한 이 범죄는 ‘2014년 이 공단 현직 이사장이 비리혐의로 수사를 받던 중 투신(사망)’하였고, ‘당시 새누리당 국회의원 2명이 비리혐의로 각각 4년 및 7년의 징역형(실형)이 선고’되는 등 이 공단이 비리의 복마전이기 때문에 가능한 것이고,또한 ‘국토교통부 등 정부관계기관 모두가 이 범죄를 의도적으로 외면’(사실상 비호)할 때 가능한 범죄이므로 김학의 전 법무부차관 비리혐의 사건과 조금도 다를 바 없습니다.그러므로 아래에서 보듯이 ‘2019. 4. 10. 국토교통부가 갑자기 태도가 돌변하여 범죄를 또다시 의도적으로 외면’하는 등 지금도 정부관계기관 모두가 이 공단 적폐를 외면하고 있어 이는 또 다른 적폐이므로, ‘검찰의 강력하고 엄정한 수사로 이를 척결’하여 이 나라 밝은 미래와 희망을 국민에게 안겨준다면 이는 정부가 국민에게 줄 수 있는 가장 큰 선물입니다.아   래 1.  이 공단을 지도·감독하는 국토교통부는 이 범죄를 고발하는 민원조차 이 공단에 이첩하고 ‘이 공단이 말도 안되는 변명만으로 종결하는 수

1.청원글번호:  579607 

2.청원제목:  억울하게 돌아가신 저희 아버지를 도와주세요 

3.참여인원:  546 명 

4.카테고리:  기타 

5.청원시작일:  2019-04-17 

6.청원마감일:  2019-05-17 

7.청원내용:  억울하게 돌아가신 저희 아버지를 살려주세요2018년 6월 18일 아침 7시경 출근하시는 아버지를 준비한 칼로 찔러 무참히 살해당했습니다.저는 평범한 직장인 20대 중반의 여성입니다.오래전부터 운동을 취미로 삼아 왔던 저는 제가 다니던 헬스장에 어린 트레이너가 새로 왔었습니다.어린 친구라 그런지 열정도 남달랐고 열심히 자신의 일에 꾸준히 하는 모습에 대견하기도 하면서 안쓰러웠고나이가 비슷하다는 이야기를 듣고 서로 좋게 잘 지내게 되었습니다.힘든 일이 있으면 저에게 이야기도 하며 엄마이름이 제 이름과 같다는 점 어릴 때부터 힘들었던 일 등등저에게 서슴없이 이야기를 하는 모습에 친동생처럼 잘 챙겨주었습니다.하지만 어느 순간부터 그는 저에게 이상한 집착과 협박을 하게 되었고 심하면 욕과 잦은 폭력을 행사였습니다.무섭고 걱정스러운 맘에 저는 그를 멀리하였고 그럴수록 그는 휴대폰으로 연락을 하거나 집을 알아와 더 심한 집착과 협박을 하였습니다.결국 가해자는 2018년 6월 18일 술을 먹고 저희 집으로 찾아와 준비한 칼을 들고 제가 출근  하기 까지 기다리고 있었답니다.그리고 먼저 출근하신 아버지를 발견하고 칼로 몇 차례 찔러 무참히 살해당하셨습니다.하지만 저희 아버지 그 아픈 몸을 이끄시고 저희 집으로 들어오려는 가해자를 붙잡고 저희 가족을 끝까지 지켜주셨습니다.칼은 저희 집 현관문 안에서 발견이 되었고 가해자는 저희 집 거실까지 들어와 저를 죽이려고 하였습니다.집 안 여기저기 피가 튀었고 그 거대한 체구로는 저희 가족들은 이길 수 없었습니다.결국 저희 가족들은 그에게 여러 차례 맞았고 피와 멍으로 온 몸에 상처가 남았었습니다.경찰에 신고를 하였을 때 그는 피로 물들은 옷을 벗고 도망을 가였고 경찰과 구급대원이 도착하였을 

...

1.청원글번호:  579597 

2.청원제목:  이게 학교인가 

3.참여인원:  294 명 

4.카테고리:  육아/교육 

5.청원시작일:  2019-04-17 

6.청원마감일:  2019-05-17 

7.청원내용:  학교에 선생님들은 등교시간 교문앞 지도를 하는데 그때 체육복을 입은학생에게 욕은 기본이고 멱살을 잡는 행위또는 학생에게 폭력을 가하는 행위들은 해왔으며 현재 3학년인 학생중 한명이 작년쯤  몰라카메라를 하였지만 생지부 쪽에서 그냥 덮고 넘어갔다고한다. 하지만 이번에도 이일이 또 생겨났으며 이번에도 그냥 넘어가려고 한다는 이야기까지 들려오는 상황이다. 이것이 만약 교육정책이라면 그 것에 바꿀필요가 있으며 학생들또한 이런 범죄들이 무서워서 교복(치마)를 어떻기 입고 다닐것인가 우리는 잘 모르겠다는 의견이고 학교도 안전하지않다는 이야기이며 학교에서도 몰래카메라 관리가 되지않으면서 사회에서는 어떻게 책임을 물을것인가, 도대체 학교라는곳에서는 안전을 가르치면서도 안전을 실천하지않는다는 근거이기도 하다 



36 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  579596 

2.청원제목:  벚꽃대신 무궁화를 살려시민의식을 높입시다. 

3.참여인원:  203 명 

4.카테고리:  기타 

5.청원시작일:  2019-04-16 

6.청원마감일:  2019-05-16 

7.청원내용:  이맘때쯤(4월) 주위를 돌아보면 온통 벚꽃 나무들 뿐입니다. 심지어 여의도 및 각 구청에서는 자체적으로 벚꽃 축제를 개최하고 있습니다.하지만, 정작 우리의 국화인 무궁화를 찾아볼 수 없는 것이 현실입니다.초등학교, 중학교 자녀들에게 국화가 무엇인지 아니? 무궁화를 본적이 있니? 그려볼래? 라고 하면 과연 몇 명이나 봤다고 대답하고, 그릴 수 있을까요?성인들조차도 무궁화를 잊고 있는 것이 현실입니다. 청와대에 가면 무궁화를 볼 수 있을지 그것도 의문입니다.더 이상 두고볼수만은 없습니다. 1회성의 단발성 광화문 

1.청원글번호:  579590 

2.청원제목:  동물학대범 **대 이**교수를 조사 및 처벌해주세요 

3.참여인원:  1,088 명 

4.카테고리:  보건복지 

5.청원시작일:  2019-04-16 

6.청원마감일:  2019-05-16 

7.청원내용:  https://n.news.naver.com/article/056/0010691553이 기사를 읽어보세요[리포트]움푹 패인 허리, 앙상하게 드러난 갈비뼈.허겁지겁 사료를 먹어치우더니 코피를 쏟기도 합니다.[검역본부 관계자/음성변조 : "아이구, 다리에 힘이 없어. 나 이런 개 처음 봐."]낮은 돌턱도 오르지 못하고 생식기는 비정상적으로 커져 있습니다.목줄에 묶인 채 러닝머신을 쉼 없이 뛰는 복제견.힘에 부치는지 뒤로 밀려났다가도, 이내 다시 끌려옵니다.[담당 수의사/음성변조 : "완전 응급이었어요. 애가 거의 막 눈이 완전 다 뒤집혀서 발작을 해서 왔기 때문에. 혹시 이거 스트레스 때문에 그런 거 아니냐 이런 얘기는 있었는데..."]지금은 마약 성분의 약을 먹으며, 하루하루를 버티고 있습니다.취재팀이 입수한 이** 교수팀의 메이의 실험 계획서입니다.'번식학 및 생리학적 정상성' 분석 실험.여러 차례 정자를 채취하고 교배를 시킨 것으로 추정됩니다.또 다른 복제견 '동'이는 오전엔 러닝머신을 뛰고, 오후에는 검역 일을 하는 일정을 소화해 왔습니다.이 교수에게 여러 차례 입장을 물었지만 묵묵부답.[이**/교수 : "(메이라는 개가 검역본부에 맡겨졌을 때 상태가 많이 안 좋았던 걸로 아는데, 개가 지금 자연사를 한 건가요?) ……."]**대는 동물 실험을 위해 검역본부에서 데려간 복제견 3마리 가운데 1마리는 죽었고, 나머지 두 마리에 대해선 여전히 실험을 진행하고 있다고 밝혔습니다.이런 미친 교수를 당장 조사하고 동물실험을 중지시키고 교수같지도 않은 교수를 처벌해주세요 부탁합니다 청원부탁드려요 [본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다] 



43 번째 국민청원 

1.청원글번호:  579584 

2.청원제목:  EBS 반민특위 다큐멘터리 제작 중단에 책임이 있는 인사의 EBS 부사장 임명 철회와 EBS의 반민특위 다큐멘터리 제작 재개를 요구합니다 

3.참여인원:  1,746 명 

4.카테고리:  문화/예술/체육/언론 

5.청원시작일:  2019-04-16 

6.청원마감일:  2019-05-16 

7.청원내용:  저는 전 EBS PD 김**입니다. 현재는 ******학교 교수로 재직 중에 있습니다. 제가 굳이 전 직장인 EBS의 부사장 임명 철회를 요구하는 이유는 이번에 새로 임명된 신임 부사장이 2013년 당시 반민특위를 다룬 다큐멘터리 제작 중단에 책임이 있기 때문입니다. 2013년 약 1년 여간 제작이 진행되던 ‘나는 독립유공자 후손입니다’라는 제목의 다큐멘터리는 담당 연출이던 제가 갑자기 수학 교육팀으로 발령이 나는 바람이 중단이 되었습니다. 인사 발령에 대한 항의로 잠시 원 제작 부서로 파견 명령이 나기도 하였으나 곧 다시 수학교육팀으로 발령이 나며 결국 제작은 완전히 중단이 되고 말았습니다. 바로 그 때 제작을 총괄하던 제작 본부장이 이번에 새로 임명된 부사장입니다. 박근혜 정권 당시 일어난 방송사 제작 자율성 침해와 역사 관련 아이템들의 제작 중단은 굳이 다시 설명을 하지 않아도 잘 알고 계시리라 생각 합니다. 저는 당시 이야기를 반복하고자 함이 아닙니다. 문제는 정권이 바뀐 상태에서 당시 이러한 문제에 책임이 있는 인사가 EBS라는 공영방송의 부사장으로 새롭게 임명이 되었다는 사실입니다. 신임 EBS 부사장은 당시 반민특위 다큐멘터리 중단에 대해 공개적인 사과를 한 적도, 제작 재개를 통해 정상화를 약속한 적도 없습니다. 그저 담당 연출이었던 제게 개인적인 사과를 한 것 뿐입니다. 개인적 사과로 넘어갈 수 있는 일이었다면 굳이 제가 이 곳에 청원글을 올릴 이유는 없을 것입니다. 당시 제가 당한 부당한 인사는 부차적 문제입니다. 본질은 이 다큐멘터리의 내용과 의미, 무엇보다 반민특위 후손이신 당사자

1.청원글번호:  579579 

2.청원제목:  제 아들을 돌려주세요...(양육권 불이행) 

3.참여인원:  309 명 

4.카테고리:  육아/교육 

5.청원시작일:  2019-04-15 

6.청원마감일:  2019-05-15 

7.청원내용:  저는 2014년 6월에 이혼하고 부모님과 함께 현재 8아이를 키워온 아버지 입니다.이혼 당시 친권은 공동, 양육권은 제가, 양육비 안받고 면접교섭권은 2주에 한번 총 48시간으로 하여 합의 이혼을 하였습니다.그리고 2년 후(2016년 12월) 아이 엄마가 면접교섭권 이행 후 아이를 약 5개월 가량 보내지 않고 아이를 저에게 잘 보여주지도 않았습니다. 우여곡절 끝에 다시 아이를 데려 왔고 다시는 이런일이 없도록 약속을 하고 다시 아이를 보여 주었습니다.(법적으로 문제삼지 않았습니다.) 그리고 2019년 4월 6일에 아이를  데려가 또 다시 보내지 않고 있습니다. 현재 아이는 학교도 보재지 않고 아이가 아프다는 핑계를 대며 보내지 않고 있습니다.경찰에 신고도 해보았으나 민사소송을 해야 한다고 만하고 경찰에서 관여 할 수 없는 문제라고 합니다.아이는 저와 있으며 큰 문제 없이 지냈으며 학교에서도 적응을 잘하고 있다고 선생님이 칭찬도 해주셨는데'아이가 학교를 가기 싫어한다.' '아빠한테 가기 싫어 한다' '아이가 아파 보낼수 없다' 이런 말을 하면서 보내지도 않고 제가 보러 가면 피하고 전화도 잘 받지 않습니다.문제는 분명 양육권은 저한테 있고 제가 양육하면서 별다를 문제가 있지도 않은데 막무가내로 보내지 않는데경찰이 개입할수도 없고 민사소송을 해도 시간이 매우 오래 걸리고 아이가 입는 상처도 무시 못한다고 하는데왜 이렇게 되는거죠? 이건 납치 아닌가요? 이럴 거면 뭐하러 합의 이혼하고 친권자 양육권자를 나누는 거죠?아이 데리고 먼저 도망가는 사람이 양육권자고 친권자 되는거 아닌가요? 시간이 흐를수록 아이는 누군가에게 계속 세뇌를 당하겠죠...(아빠랑 같이가면 안되)대체 이런 법이 법인가요? 정말 답답하고 화가납니다

1.청원글번호:  579575 

2.청원제목:  CRPS (복합부위통증증후군) 환우분들을 도와주세요. 

3.참여인원:  225 명 

4.카테고리:  기타 

5.청원시작일:  2019-04-15 

6.청원마감일:  2019-05-15 

7.청원내용:  안녕하십니까 저는 아주 평범한 고등학생입니다.제가 최근 sns를 통해 crps (복합부위통증증후군)이라는 병에 대하여 알게 되었습니다. 이 병을 알고 난 뒤 이 병에 대해 관심을 가지고 찾아보기 시작했습니다. 이병은 가벼운 골절이나 부상으로 인해 알 수 없는 심각한 고통, 부종, 피부의 변화를 수반하는 만성 통증 질환입니다.주로 팔이나 다리에 영향을 미치지만 몸의 다른 부위로도 퍼져나갈 수 있으며, 시간이 지날수록 악화되는 경향이 있습니다.이 병은 바람만 불어도 작열통 즉 불에 타는듯한 고통을 느끼며 두꺼운 작도 칼로 살을 썰어내는 고통을 많게는 24시간 동안이나 느끼며 고통스러워합니다.마약성 진통제를 써도 어떠한 방법을 써도 고통은 불시에 찾아옵니다.CRPS 환자 824명 중에서 92%는 불면증을, 78%는 흥분, 불안, 분노 등의 감정조절 장애를, 73%는 우울증을, 그리고 48%는 기억력과 집중력 저하를 호소한다고 합니다. 이분들은 대체 무슨 잘못을 하신 걸까요왜 이렇게 억울하게 아파야 하는 걸까요아픈 것도 모자라 너무 가슴 아프게도이 병을 겪고 있는 환우분들을 나라에선 장애인 판정도 해주지 않으며조금의 지원도 해주지 않고 있습니다.이 병을 겪고 계시는 환우분들의 가족들은 어마어마한 병원비와 약 값을 벌기 위해 하루하루 힘들게 살아가십니다. 또 상상 이상으로 돈이 많이 들기 때문에 아파하는 환우분들은 보며 정신적인 고통도 매우 크실 겁니다. 이 병은 드물지만 누구나 걸릴 수 있는 병이며 아주 무섭고 사악한 병입니다. 억울하게 병에 걸린 환우 분들과 그 가족분들이 나라에서 조금의 지원도 받지 못하고 있다는 사실에 너무 가슴이 아프고 눈물밖에 나오지 않았습니다. 그래서 저는 청원 글을 올려야겠다는 생각